In [ ]:
# :: 10th January 2023

SOLID: Open / closed principle:
- Software entities (classes, modules, methods, etc.)
  - Should be OPEN for extension but
  - Should be CLOSED for modification.

OCP violation Problems:
- Classes are tightly couples
  - Changing one bit of code affects multiple entities if you have tight coupling

- Code is difficult to test in isolation
  - As it is hard to seperete code if it is tightly coupled.

Spotting OCP violations:
- Does a class know too much about its dependancies?
  - Say you have a class that has an attribute which is an external objects
    - if that class knows too much about that objects strategy which it uses to carry out a certain job then you violate the OCP principle.

In [1]:
class Extractor: # 
    def extract_spectrogram(self, data):
        print(f'Extracted spectrogram')
        
    def extract_mfcc(self, data):
        print(f'Extracted MFCCs')

class DLPipeline:
    def __init__(self, extractor, feature_type):
        self.extractor = extractor
        self.feature_type = feature_type
    
    def run(self, data):
        print(f'Running DL pipeline')
        features = self._extract(data) # Private method
        # Here implementation of DL steps
        
    def _extract(self, data):
        if self.feature_type == 'spectrogram':
            self.extractor.extract_spectrogram(data)
        elif self.feature_type == 'mfcc':
            self.extractor.extract_mfcc(data)
            
if __name__ == '__main__':
    data = [1,2,3]
    extractor = Extractor()
    dl_pipeline = DLPipeline(extractor, 'spectrogram')
    dl_pipeline.run(data)

Running DL pipeline
Extracted spectrogram


DLPipeline owns Extractor (relationship)

Extractor has 2 methods (mfcc extract and spectrogram extraction)
-   If we wanted to add another feature such as mel spectrogram extraction
    -   we would have to modify two entities rather than adding a new function.

### DESIGN that violates open close principle

In [ ]:
class Extractor: # 
    def extract_spectrogram(self, data):
        print(f'Extracted spectrogram')
        
    def extract_mfcc(self, data):
        print(f'Extracted MFCCs')
        
    def extract_mel_spectrogram(self, data): # and add this
        print('Extracted mel spectrogram')

class DLPipeline:
    def __init__(self, extractor, feature_type):
        self.extractor = extractor
        self.feature_type = feature_type
    
    def run(self, data):
        print(f'Running DL pipeline')
        features = self._extract(data) # Private method
        # Here implementation of DL steps
        
    def _extract(self, data):
        if self.feature_type == 'spectrogram':
            self.extractor.extract_spectrogram(data)
        elif self.feature_type == 'mfcc':
            self.extractor.extract_mfcc(data)
        elif self.feature_type == 'melspectrogram': # need to add this
            self.extractor.extract_mel_spectrogram(data)
            
if __name__ == '__main__':
    data = [1,2,3]
    extractor = Extractor()
    dl_pipeline = DLPipeline(extractor, 'spectrogram')
    dl_pipeline.run(data)

Above violates open close principle as for adding a new functionality we must change the code in 2 places.

Does DL pipeline know too much about extractor? Yep.
- Extract private method in extractor the dl pipeline knows too much about what extractor does as the dl pipeline has to know this.

We can solve this using polymorphism
- Extractor becomes an interface that will have a single method extract that only takes data.
- We add concrete classes which inherit from extracvtor and implement the extract class to add more degrees of seperation between all the sub extraction classes.
- This makes the extraction classes now implementations aka subclasses / concrete abstractors of extractor interface using different strategies.

### Design that respects OPEN CLOSE principle

In [4]:
from abc import abstractmethod, ABC


class Extractor(ABC): # 
    @abstractmethod
    def extract(self, data):
        pass
    
class SpectrogramExtractor(Extractor):
    def extract(self, data):
        print(f'Extracted spectorgram')
    
class MFCCExtractor(Extractor):
    def extract(self, data):
        print(f'Extracted MFCCs')
        
class MelSpectrogramExtractor(Extractor):
    def extract(self, data):
        print('Extracted mel spectrogram')

        

class DLPipeline:
    def __init__(self, extractor):
        self.extractor = extractor
    
    def run(self, data):
        print(f'Running DL pipeline')
        features = self._extract(data) # Private method
        # Here implementation of DL steps
        
    def _extract(self, data): # New design does not have access to the strategies in extractor. It just knows the interface.
        self.extractor.extract(data)
            
if __name__ == '__main__':
    data = [1,2,3]
    extractor = SpectrogramExtractor()
    dl_pipeline = DLPipeline(extractor)
    dl_pipeline.run(data)

Running DL pipeline
Extracted spectorgram
